In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from torch.utils.data import IterableDataset
from torch.optim.lr_scheduler import ExponentialLR
from transformers import (BertTokenizer,
                          BertModel, 
                          BertForMaskedLM, 
                          BertForNextSentencePrediction, 
                          AdamW)

from dataset import *
from utils import *

In [3]:
device = 'cuda'
parallel = True
ckpt_file = 'bert-wikihow.pt'
save_ckpt = False

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
collator = NSPBatchCollator(tokenizer, device=device)

In [6]:
dataset = WikihowNSP('wikihow.csv', shuffle=True, tqdm=tqdm_notebook)

In [12]:
loader = dataset.loader(collator, batch_size=32)

In [13]:
raw_model = (BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
             if ckpt_file is None
             else torch.load(ckpt_file))
model = nn.DataParallel(raw_model.to(device)) if parallel else raw_model

In [14]:
optimizer = AdamW(model.parameters(), lr=1e-6, weight_decay=0.01)

In [15]:
scheduler = ExponentialLR(optimizer, gamma=0.999)

In [11]:
def reduce_metrics(seq):
    accuracy, loss, lr = zip(*seq)
    racc = f'{np.mean(accuracy) * 100:.2f}%'
    rloss = f'{np.mean(loss):.2e}'
    rlr = f'{lr[-1][0]:.2e}'
    return f'acc: {racc}, loss: {rloss}, lr: {rlr}'

In [12]:
with Logger(file='train.log', 
            reduce_fn=reduce_metrics, 
            header=('=' * 50),
            overwrite=False, 
            period=50) as logger:
    for epoch in range(8):
        for it, (input_ids, 
                 token_type_ids, 
                 attention_mask, 
                 next_sentence_label) in enumerate(loader):

            model.train()
            model.zero_grad()
            loss, score = model(input_ids=input_ids, 
                                token_type_ids=token_type_ids, 
                                attention_mask=attention_mask, 
                                next_sentence_label=next_sentence_label)
            next_sentence_pred = score.argmax(dim=1)
            accuracy = (next_sentence_pred == next_sentence_label).float().mean()
            loss.mean().backward()
            logger.step([accuracy.detach().cpu().numpy(),
                         loss.mean().detach().cpu().numpy(), 
                         scheduler.get_lr()])
            optimizer.step()
            scheduler.step()

            if save_ckpt and it % 100 == 0:
                torch.save(raw_model, ckpt_file)

KeyboardInterrupt: 

In [17]:
with Logger(file='eval.log', 
            reduce_fn=np.mean, 
            header=('accuracy'),
            overwrite=True, 
            period=50) as logger:
    for epoch in range(8):
        for it, (input_ids, 
                 token_type_ids, 
                 attention_mask, 
                 next_sentence_label) in enumerate(loader):

            model.eval()
            loss, score = model(input_ids=input_ids, 
                                token_type_ids=token_type_ids, 
                                attention_mask=attention_mask, 
                                next_sentence_label=next_sentence_label)
            next_sentence_pred = score.argmax(dim=1)
            accuracy = (next_sentence_pred == next_sentence_label).float().mean()
            logger.step(accuracy.detach().cpu().numpy())

KeyboardInterrupt: 